In [1]:
import common.pagerank as pr
import numpy as np

In [2]:
adj_matrix = np.array([[0, 1, 1, 0], [1, 0, 0, 1], [1, 0, 0, 0], [0, 1, 1, 0]])

# Define multiple parameter configurations to explore
parameter_configs = [
    {"d": 0.85, "max_iter": 100000},
    {"d": 0.9, "max_iter": 100000},
    {"d": 0.95, "max_iter": 100000},
]

# Explore parameter configurations
results = pr.explore_parameter_configurations(adj_matrix, parameter_configs)

# Print results
for config, values in results.items():
    print("Parameter Configuration:", config)
    print("PageRank scores:", values[0])
    print("Number of steps to convergence:", values[1])
    print()

c:\Users\Dion\anaconda3\envs\cloud2\Lib\site-packages\numpy\core\fromnumeric.py:88: RuntimeWarning: overflow encountered in reduce
  return ufunc.reduce(obj, axis, dtype, out, **passkwargs)
e:\Github\Year 4\CZ4052-Cloud-Computing-Assignment-2\common\pagerank.py:29: RuntimeWarning: invalid value encountered in multiply
  incoming_score = np.sum(adj_matrix[:, i] * prev_pagerank_scores)


DEBUG: 100000
DEBUG: 100000
DEBUG: 100000
Parameter Configuration: {'d': 0.85, 'max_iter': 100000}
PageRank scores: [nan nan nan nan]
Number of steps to convergence: 100000

Parameter Configuration: {'d': 0.9, 'max_iter': 100000}
PageRank scores: [nan nan nan nan]
Number of steps to convergence: 100000

Parameter Configuration: {'d': 0.95, 'max_iter': 100000}
PageRank scores: [nan nan nan nan]
Number of steps to convergence: 100000

